In [1]:
!pip install ultralytics -q

# 2. GPU Kontrolü
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU Aktif: {gpu_name}")
    # T4 belleğini temizle
    torch.cuda.empty_cache()
else:
    raise Exception("❌ GPU bulunamadı! Runtime Type'ı 'T4 GPU' seçmelisin.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.8 MB/s eta 0:00:00
✅ GPU Aktif: Tesla T4


In [2]:
import os
import shutil
import zipfile
import gdown

# --- AYARLAR ---
FILE_ID = '1kSFIepq6LkqUsZq9tyExSk7pYwkc8aw8' 
OUTPUT_ZIP = 'dental_ultimate.zip'
DATASET_DIR = '/content/dataset'

# İndirme
if not os.path.exists(OUTPUT_ZIP):
    print(f"⬇️ YOLO8 için veri seti indiriliyor...")
    gdown.download(id=FILE_ID, output=OUTPUT_ZIP, quiet=False, fuzzy=True)

# Çıkarma
if os.path.exists(OUTPUT_ZIP) and os.path.getsize(OUTPUT_ZIP) > 1000:
    print("📂 Zip açılıyor...")
    if os.path.exists(DATASET_DIR): shutil.rmtree(DATASET_DIR)
    
    with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zip_ref:
        zip_ref.extractall(DATASET_DIR)
        
    # YAML Güncelleme
    yaml_path = f"{DATASET_DIR}/processed_ultimate/data.yaml"
    new_yaml = f"""
train: {DATASET_DIR}/processed_ultimate/train/images
val: {DATASET_DIR}/processed_ultimate/val/images
nc: 4
names:
  0: Impacted
  1: Caries
  2: Periapical Lesion
  3: Deep Caries
"""
    with open(yaml_path, 'w') as f: f.write(new_yaml)
    print("✅ Veri ve Ayarlar Hazır.")
else:
    print("❌ İndirme Hatası. Drive izinlerini kontrol et.")

⬇️ YOLO8 için veri seti indiriliyor...


Downloading...
From (original): https://drive.google.com/uc?id=1kSFIepq6LkqUsZq9tyExSk7pYwkc8aw8
From (redirected): https://drive.google.com/uc?id=1kSFIepq6LkqUsZq9tyExSk7pYwkc8aw8&confirm=t&uuid=ad313a90-0de8-4aa6-8344-7a7efe6f7acd
To: /content/dental_ultimate.zip
100%|██████████| 915M/915M [00:08<00:00, 108MB/s]  


📂 Zip açılıyor...
✅ Veri ve Ayarlar Hazır.


In [3]:
from ultralytics import YOLO
import torch

# GPU Kontrol
if not torch.cuda.is_available():
    raise Exception("❌ GPU Yok! Runtime Type 'T4 GPU' olmalı.")

# 1. Modeli Başlat (YOLOv8 Large)
model = YOLO('yolov8l.pt') 

print("🚀 ULTIMATE EĞİTİM BAŞLIYOR (1280px + Large Model)...")

# 2. Eğitimi Başlat
results = model.train(
    data='/content/dataset/processed_ultimate/data.yaml',
    project='/content/runs', # Geçici olarak Colab diskine kaydet
    name='v6_ultimate_large',
    
    # --- Donanım Limitleri (T4 16GB) ---
    epochs=100,
    imgsz=1280,           # Yüksek Çözünürlük (Küçük çürükler için)
    batch=4,              # T4 Belleği için güvenli sınır. (Hata alırsan 2 yap)
    device=0,
    workers=4,
    
    # --- Optimizasyon ---
    patience=20,
    optimizer='AdamW',
    lr0=0.0005,           # Büyük model için düşük LR
    cos_lr=True,
    
    # --- Augmentation (Full Paket) ---
    mosaic=1.0, 
    mixup=0.2,            # Karıştırma
    copy_paste=0.2,       # Nesne yapıştırma (Veri çoğaltma)
    degrees=20.0,         # Döndürme
    translate=0.1,
    scale=0.7,            # Zoom in/out
    fliplr=0.5,
    
    save=True,
    exist_ok=True,
    verbose=True
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 ULTIMATE EĞİTİM BAŞLIYOR (1280px + Large Model)...
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset/processed_ultimate/data.yaml, degrees=20.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.

In [ ]:
import shutil
from google.colab import files

print("📦 Sonuçlar paketleniyor...")
shutil.make_archive('training_results', 'zip', '/content/runs/v6_ultimate_large')

print("⬇️ İndirme başlıyor...")
files.download('training_results.zip')

📦 Sonuçlar paketleniyor...
⬇️ İndirme başlıyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>